In [24]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action="ignore", category=FutureWarning)

In [25]:
pizza = pd.read_csv("Pizza_sales_dataset_proper.csv")
print(pizza.head())

   Unnamed: 0  pizza_id  order_id  pizza_name_id  quantity  order_date  \
0           0         1         1     hawaiian_m         1  01-01-2015   
1           1         2         2  classic_dlx_m         1  01-01-2015   
2           2         3         2  five_cheese_l         1  01-01-2015   
3           3         4         2    ital_supr_l         1  01-01-2015   
4           4         5         2     mexicana_m         1  01-01-2015   

  order_day order_time  unit_price  total_price pizza_size pizza_category  \
0  Thursday   11:38:36       13.25        13.25          M        Classic   
1  Thursday   11:57:40       16.00        16.00          M        Classic   
2  Thursday   11:57:40       18.50        18.50          L         Veggie   
3  Thursday   11:57:40       20.75        20.75          L        Supreme   
4  Thursday   11:57:40       16.00        16.00          M         Veggie   

                                   pizza_ingredients  \
0           Sliced Ham, Pineapple, M

In [28]:
pizza['quantity'].value_counts()

quantity
1    47693
2      903
3       21
4        3
Name: count, dtype: int64

In [27]:
pizza['quantity'].unique()

array([1, 2, 3, 4])

In [4]:
pizza.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         48620 non-null  int64  
 1   pizza_id           48620 non-null  int64  
 2   order_id           48620 non-null  int64  
 3   pizza_name_id      48620 non-null  object 
 4   quantity           48620 non-null  int64  
 5   order_date         48620 non-null  object 
 6   order_day          48620 non-null  object 
 7   order_time         48620 non-null  object 
 8   unit_price         48620 non-null  float64
 9   total_price        48620 non-null  float64
 10  pizza_size         48620 non-null  object 
 11  pizza_category     48620 non-null  object 
 12  pizza_ingredients  48620 non-null  object 
 13  pizza_name         48620 non-null  object 
dtypes: float64(2), int64(4), object(8)
memory usage: 5.2+ MB


In [3]:
pizza.describe()

,Unnamed: 0,pizza_id,order_id,quantity,unit_price,total_price
count,48620.000000,48620.000000,48620.000000,48620.000000,48620.000000,48620.000000
mean,24309.500000,24310.500000,10701.479761,1.019622,16.494132,16.821474
std,14035.529381,14035.529381,6180.119770,0.143077,3.621789,4.437398
min,0.000000,1.000000,1.000000,1.000000,9.750000,9.750000
25%,12154.750000,12155.750000,5337.000000,1.000000,12.750000,12.750000
50%,24309.500000,24310.500000,10682.500000,1.000000,16.500000,16.500000
75%,36464.250000,36465.250000,16100.000000,1.000000,20.250000,20.500000
max,48619.000000,48620.000000,21350.000000,4.000000,35.950000,83.000000


In [5]:
n_totalpizzas = len(pizza)
n_totalids = len(pizza['order_id'].unique())
n_offered_pizzas = len(pizza['pizza_name'].unique())
avg_count_per_pizza = round(n_totalpizzas / n_offered_pizzas, 2)
print(f"Total number of pizzas orders: {n_totalpizzas}")
print(f"Total number of people ordering pizzas : {n_totalids}")
print(f"Total number of pizzas offered: {n_offered_pizzas}")
print(f"Average count of pizza ordered by a person: {round(n_totalpizzas/n_totalids, 2)}")
print(f"Average count of each pizza sold: {round(n_totalpizzas/n_offered_pizzas, 2)}")

Total number of pizzas orders: 48620
Total number of people ordering pizzas : 21350
Total number of pizzas offered: 32
Average count of pizza ordered by a person: 2.28
Average count of each pizza sold: 1519.38


In [ ]:
#pizza['pizza_name_id'] = pizza['pizza_name'].astype('category').cat.codes + 1

In [8]:
off_pizza_freq = pizza[['pizza_name','pizza_size','pizza_category','unit_price']].groupby(['pizza_name','pizza_size','pizza_category']).count().reset_index()
off_pizza_freq.columns = ['pizza_name','pizza_size','pizza_category','avg_count_per_pizza']
print(off_pizza_freq.head())

                   pizza_name pizza_size pizza_category  avg_count_per_pizza
0  The Barbecue Chicken Pizza          L        Chicken                  967
1  The Barbecue Chicken Pizza          M        Chicken                  926
2  The Barbecue Chicken Pizza          S        Chicken                  479
3          The Big Meat Pizza          S        Classic                 1811
4        The Brie Carre Pizza          S        Supreme                  480


In [11]:
count_sales = pizza.groupby(['pizza_name','pizza_size','pizza_category'])['quantity'].count().reset_index()
count_sales

,pizza_name,pizza_size,pizza_category,quantity
0,The Barbecue Chicken Pizza,L,Chicken,967
1,The Barbecue Chicken Pizza,M,Chicken,926
2,The Barbecue Chicken Pizza,S,Chicken,479
3,The Big Meat Pizza,S,Classic,1811
4,The Brie Carre Pizza,S,Supreme,480
...,...,...,...,...
86,The Thai Chicken Pizza,M,Chicken,474
87,The Thai Chicken Pizza,S,Chicken,476
88,The Vegetables + Vegetables Pizza,L,Veggie,423
89,The Vegetables + Vegetables Pizza,M,Veggie,630


In [14]:
lowest_sold_pizza = count_sales['quantity'].idxmin()
lowest_sold_pizza_name = count_sales.loc[lowest_sold_pizza, 'pizza_name']
lowest_sold_pizza_size = count_sales.loc[lowest_sold_pizza, 'pizza_size']
lowest_sold_pizza_category = count_sales.loc[lowest_sold_pizza, 'pizza_category']
lowest_sold_pizza_count = count_sales.loc[lowest_sold_pizza, 'quantity']
print(f"Lowest sold pizza: {lowest_sold_pizza_name} ({lowest_sold_pizza_size}, {lowest_sold_pizza_category}) with {lowest_sold_pizza_count} orders")

Lowest sold pizza: The Greek Pizza (XXL, Classic) with 28 orders


In [16]:
pizza.loc[pizza['pizza_name'] == lowest_sold_pizza_name].head()
pizza.loc[pizza['pizza_name'] == lowest_sold_pizza_name, 'quantity'].sum()
#pizza.loc[pizza['pizza_name'] == lowest_sold_pizza_name, 'quantity'].sum() / pizza['quantity'].sum()
#pizza.loc[pizza['pizza_name'] == lowest_sold_pizza_name, 'quantity'].sum() / pizza['quantity'].sum() * 100

np.int64(1420)

In [17]:
highest_sold_pizza = count_sales['quantity'].idxmax()
highest_sold_pizza_name = count_sales.loc[highest_sold_pizza, 'pizza_name']
highest_sold_pizza_size = count_sales.loc[highest_sold_pizza, 'pizza_size']
highest_sold_pizza_category = count_sales.loc[highest_sold_pizza, 'pizza_category']
highest_sold_pizza_count = count_sales.loc[highest_sold_pizza, 'quantity']
print(f"Highest sold pizza: {highest_sold_pizza_name} ({highest_sold_pizza_size}, {highest_sold_pizza_category}) with {highest_sold_pizza_count} orders")

Highest sold pizza: The Big Meat Pizza (S, Classic) with 1811 orders


In [23]:
pizza_stats = pizza.groupby(['pizza_name', 'pizza_size', 'pizza_category'])[['total_price']].agg(['sum','mean'])
pizza_stats.columns = pizza_stats.columns.droplevel()

pizza_stats

sum  \
pizza_name                        pizza_size pizza_category             
The Barbecue Chicken Pizza        L          Chicken         20584.00   
                                  M          Chicken         16013.00   
                                  S          Chicken          6171.00   
The Big Meat Pizza                S          Classic         22968.00   
The Brie Carre Pizza              S          Supreme         11588.50   
...                                                               ...   
The Thai Chicken Pizza            M          Chicken          8056.75   
                                  S          Chicken          6120.00   
The Vegetables + Vegetables Pizza L          Veggie           8646.75   
                                  M          Veggie          10160.00   
                                  S          Veggie           5568.00   

                                                                  mean  
pizza_name                        pizza_size pizza_category             
The Barbecue Chicken Pizza        L          Chicken         21.286453  
                                  M          Chicken         17.292657  
                                  S          Chicken         12.883090  
The Big Meat Pizza                S          Classic         12.682496  
The Brie Carre Pizza              S          Supreme         24.142708  
...                                                                ...  
The Thai Chicken Pizza            M          Chicken         16.997363  
                                  S          Chicken         12.857143  
The Vegetables + Vegetables Pizza L          Veggie          20.441489  
                                  M          Veggie          16.126984  
                                  S          Veggie          12.183807  

[91 rows x 2 columns]

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import streamlit as st

# Load data
df = pd.read_csv("Pizza_sales_dataset_proper.csv")

# Composite pizza item name
df['pizza_item'] = df['pizza_category'] + "_" + df['pizza_size'] + "_" + df['pizza_name']

# Use total_price as a signal of preference
pizza_data = df[['order_id', 'pizza_item', 'total_price']].copy()
pizza_data.columns = ['order_id', 'pizza_item', 'total_price']



In [38]:
def create_matrix(df):
    N = len(df['userId'].unique())
    M = len(df['itemId'].unique())

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
    item_mapper = dict(zip(np.unique(df["itemId"]), list(range(M))))

    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
    item_inv_mapper = dict(zip(list(range(M)), np.unique(df["itemId"])))

    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [item_mapper[i] for i in df['itemId']]

    X = csr_matrix((df["rating"], (item_index, user_index)), shape=(M, N))
    return X, item_mapper, item_inv_mapper

X, item_mapper, item_inv_mapper = create_matrix(pizza_data)



In [39]:
def find_similar_pizzas(selected_pizza, X, k=5, metric='cosine'):
    if selected_pizza not in item_mapper:
        print(f"Pizza '{selected_pizza}' not found!")
        return []

    index = item_mapper[selected_pizza]
    pizza_vec = X[index]

    if pizza_vec.nnz == 0:
        print(f"No data available for '{selected_pizza}'")
        return []

    kNN = NearestNeighbors(n_neighbors=min(k + 1, X.shape[0]), algorithm='brute', metric=metric)
    kNN.fit(X)
    pizza_vec = pizza_vec.reshape(1, -1)

    try:
        neighbors = kNN.kneighbors(pizza_vec, return_distance=False).flatten()
    except:
        print("Failed to find neighbors")
        return []

    neighbors = [item_inv_mapper[i] for i in neighbors if i != index]
    return neighbors[:k]



In [41]:
def recommend_items_for_user(user_id, X, k=5):
    if user_id not in user_mapper:
        print("User not found!")
        return

    user_idx = user_mapper[user_id]
    user_ratings = X[:, user_idx].toarray().flatten()

    if np.all(user_ratings == 0):
        print("This user hasn't bought anything yet!")
        return

    top_item_idx = np.argmax(user_ratings)
    top_item_id = item_inv_mapper[top_item_idx]

    print(f"Since you ordered '{top_item_id}', you might also like:")
    similar_items = find_similar_items(top_item_id, X, k)

    if not similar_items:
        print("Sorry, no similar pizzas found.")
        return

    for item in similar_items:
        print(f"- {item}")


In [40]:
# Pick a pizza from the unique set
example_pizza = df['pizza_item'].unique()[0]  # or let user choose

print(f"Because you selected: {example_pizza}")
similar = find_similar_pizzas(example_pizza, X, k=5)

for i, item in enumerate(similar, 1):
    print(f"{i}. {item}")


Because you selected: Classic_M_The Hawaiian Pizza
1. Classic_S_The Big Meat Pizza
2. Chicken_L_The Barbecue Chicken Pizza
3. Chicken_M_The Barbecue Chicken Pizza
4. Veggie_L_The Five Cheese Pizza
5. Supreme_L_The Pepper Salami Pizza


In [36]:
recommend_items_for_user(user_id=2, X=X, k=5)


Since you ordered 'Chicken_L_The Thai Chicken Pizza', you might also like:
Not enough neighbors found for 'Chicken_L_The Thai Chicken Pizza'
Sorry, no similar pizzas found.
